# Canada Postal Codes Dataframe Assignment

## Applied Data Science Capstone Project - Coursera: Week 3 assignment

### Nishant Vemulakonda

### Importing Libraries

In [1]:
import pandas as pd # library for data analsysis
import numpy as np

from bs4 import BeautifulSoup as bs # Library for Web scraping
import urllib

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported')

Libraries imported


## Part 1: Web scraping for Toronto neighborhood and build a clean dataframe

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [2]:
# Get data from Wiki page using BeautifulSoup
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

soup = bs(urllib.request.urlopen(URL), 'html.parser')
print(soup.prettify()[:500])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_po


### Scrape the HTML
The data we want is in a table, with 3 columns PostalCode, Borough and Neighbourhood.

In [3]:
#Extracting column names (Table headers) for pandas dataframe
columns = [str(col.string.replace("\n","")) for col in soup.table.find_all('th')] 
print(columns)

# Extracting row data for pandas dataframe
row_text = [str(bs(str(row).replace("\n",""), "lxml").text) for row in soup.table.find_all('td')]
print(row_text[:20])

['Postcode', 'Borough', 'Neighbourhood']
['M1A', 'Not assigned', 'Not assigned', 'M2A', 'Not assigned', 'Not assigned', 'M3A', 'North York', 'Parkwoods', 'M4A', 'North York', 'Victoria Village', 'M5A', 'Downtown Toronto', 'Harbourfront', 'M5A', 'Downtown Toronto', 'Regent Park', 'M6A', 'North York']


In [4]:
postalcodes = row_text[0::3]
boroughs = row_text[1::3]
neighbourhoods =  row_text[2::3]
# Checking length of each list, i.e. number of rows
print("Length: {},{},{}".format(len(postalcodes),len(boroughs),len(neighbourhoods)))

Length: 289,289,289


### Tranform the data into a *pandas* dataframe
We will put the 3 lists created above to a pandas dataframe with corresponding columns name.

In [5]:
df = pd.DataFrame(columns=columns)
df.rename(columns={"Postcode":"Postalcode"},inplace=True)
# Looping thorugh lists created above to insert data into dataframe
df.Postalcode = postalcodes
df.Borough =  boroughs
df.Neighbourhood = neighbourhoods
print ("df dimensions:",df.shape)

# Applying filters to get relevant data 

# Filter mask to remove "not assigned" Boroughs
# Removing not assigned rows There are some postal codes which are not belongs to any borough.

filter_mask = df.Borough == "Not assigned"

#Filtering dataframe
df_filtered = df[~filter_mask]
print ("df_filtered dimensions:", df_filtered.shape)

df dimensions: (289, 3)
df_filtered dimensions: (212, 3)


### Deal with Not assigned Neighborhood
For M7A Queen's Park, there is no neighborhood assigned.
We will replace the 'Not assigned' with the value of the corresponding Borough

In [6]:
#Replacing Neighborhoods with "not assigned" with Borough value:
df_filtered.Neighbourhood[df_filtered.Neighbourhood == 'Not assigned'] = df_filtered.Borough
print ("df_filtered dimensions:",df_filtered.shape)
# df_filtered.Neighbourhood.to_csv('data.csv') #validating data

df_filtered dimensions: (212, 3)


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2862: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df_final = df_filtered.groupby(['Postalcode','Borough']).Neighbourhood.agg(lambda val: ','.join(val)).reset_index()
df_final.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### DateFrame cleaned
#### The final dataframe has 103 rows

In [8]:
print (df_final.shape)

(103, 3)


## Part 2: Getting coordinates and add to the Toronto DataFrame

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

Note: Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

The csv file format has:
3 columns: Postal Code, Latitude and Longitude
103 rows: corresponding to 103 postal codes in our toronto dataframe

In [9]:
geo_coordinates_data = pd.read_csv('http://cocl.us/Geospatial_data')
print(geo_coordinates_data.shape)
geo_coordinates_data.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Now, We will merge the two dataframes.
To get the correct result, we will need to perform an inner join on the dataframes using Postal Code column as the join key.

In [10]:
df_with_coordinates = pd.merge(df_final, geo_coordinates_data, left_on="Postalcode",right_on="Postal Code", how='inner')
df_with_coordinates.drop("Postal Code", axis=1, inplace=True)

df_with_coordinates.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
df_with_coordinates.shape

(103, 5)